In [3]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26820 sha256=1762c3bd7009d79410001f8615da77a37ada79bc2a5f06348f80cab7bc504570
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [4]:
import pandas as pd
import os
from pandasql import sqldf
import sqlite3

In [5]:
pysqldf = lambda q: sqldf(q, globals())

In [7]:
dataset=pd.read_excel("Survey_Data.xlsx")
dataset

,Respondent ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question 1-Response,Question 2-Response,Question 3-Open-Ended Response,Question 4-Response,Question 4-Other (please specify),Question 5-Response 1,Question 5-Response 2,Question 5-Response 3,Question 5-Response 4,Question 5-Response 5,Question 5-Response 6,Question 6-Response 1,Question 6-Response 2,Question 6-Response 3,Question 6-Response 4,Question 6-Response 5,Question 6-Response 6,Question 7-Response 1,Question 7-Unscheduled,Question 8-Response 1,Question 8-Response 2,Question 8-Response 3,Question 8-Response 4,Question 9-Response 1,Question 9-Response 2,Question 9-Response 3,...,Question 22-Reponse 2,Question 23-Response,Question 24-Response 1,Question 24-Response 2,Question 24-Response 3,Question 24-Response 4,Question 24-Response 5,Question 25-Response 1,Question 25-Response 2,Question 25-Response 3,Question 25-Response 4,Question 25-Response 5,Question 25-Response 6,Question 25-Response 7,Question 25-Response 8,Question 25-Response 9,Question 26-Response 1,Question 26-Response 2,Question 26-Response 3,Question 26-Response 4,Question 27-Response 1,Question 27-Response 2,Question 28-Response,Question 29-Response 1,Question 29-Response 2,Question 29-Response 3,Question 29-Response 4,Question 29-Response 5,Question 29-Response 6,Question 29-Response 7,Question 29-Response 8,Question 29-Response 9,Question 29-Response 10,Question 29-Response 11,Question 29-Response 12,Question 29-Response 13,Question 29-Response 14,Question 30-Response 1,Question 30-Response 2,Question 30-Response 3
0,5379192392,2021-01-22 12:01:17,2021-01-22 12:40:34,NaN,NaN,NaN,NaN,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,NaN,Answer 6,NaN,Answer 2,NaN,Answer 4,NaN,NaN,Answer 5,Answer 5,Answer 2,NaN,Answer 2,Answer 5,Answer 3,NaN,NaN,Answer 1,Answer 1,Answer 6,Answer 3,Answer 1,NaN,Answer 1,Answer 3,Answer 6,...,NaN,NaN,NaN,NaN,Answer 2,Answer 5,Answer 8,Answer 7,Answer 8,Answer 8,NaN,Answer 4,Answer 5,NaN,Answer 6,NaN,Answer 4,Answer 5,NaN,Answer 7,Answer 5,NaN,NaN,Answer 3,Answer 2,Answer 5,Answer 6,NaN,Answer 6,Answer 1,NaN,Answer 8,Answer 8,Answer 4,NaN,NaN,Answer 5,NaN,NaN,NaN
1,2658722536,2021-01-22 06:56:37,2021-01-22 07:34:10,NaN,NaN,NaN,NaN,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Answer 4,Answer 2,NaN,Answer 4,Answer 7,Answer 1,Answer 8,Answer 5,Answer 6,NaN,Answer 7,Answer 7,Answer 6,NaN,NaN,Answer 5,Answer 3,Answer 3,Answer 3,Answer 7,NaN,NaN,NaN,NaN,Answer 4,Answer 2,...,Answer 4,Answer 7,NaN,NaN,NaN,NaN,NaN,Answer 5,NaN,NaN,Answer 7,Answer 2,NaN,Answer 1,NaN,Answer 6,Answer 3,Answer 6,NaN,NaN,Answer 1,NaN,NaN,NaN,Answer 4,Answer 2,NaN,Answer 5,Answer 4,Answer 1,NaN,Answer 5,NaN,NaN,Answer 2,NaN,Answer 5,NaN,NaN,Answer 1
2,4044163394,2021-01-22 06:35:18,2021-01-22 06:47:32,NaN,NaN,NaN,NaN,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Answer 5,Answer 7,NaN,Answer 8,Answer 1,Answer 5,NaN,NaN,Answer 4,NaN,Answer 7,Answer 8,Answer 5,Answer 4,Answer 8,Answer 7,Answer 2,Answer 7,Answer 1,NaN,Answer 1,Answer 8,Answer 5,Answer 2,Answer 6,NaN,...,NaN,Answer 3,Answer 1,Answer 2,NaN,NaN,Answer 6,Answer 4,NaN,NaN,NaN,Answer 7,Answer 3,Answer 5,Answer 5,Answer 7,NaN,Answer 7,Answer 1,Answer 7,Answer 4,NaN,Answer 4,NaN,Answer 7,NaN,Answer 3,Answer 8,Answer 3,Answer 7,NaN,NaN,Answer 4,Answer 4,Answer 6,NaN,Answer 6,NaN,Answer 1,NaN
3,5535865599,2021-01-21 21:29:32,2021-01-21 21:40:24,NaN,NaN,NaN,NaN,Infrastructure,NaN,Manager,Millenni

In [10]:
dataset_modified=dataset.copy() #creating a copu of the original dataframe

In [12]:
drop_col=['Start Date', 'End Date', 'Email Address','First Name', 'Last Name', 'Custom Data 1']
drop_col

['Start Date',
 'End Date',
 'Email Address',
 'First Name',
 'Last Name',
 'Custom Data 1']

In [13]:
dataset_modified.drop(columns=drop_col,axis=1,inplace=True)
dataset_modified

In [20]:
id_vars=list(dataset_modified.columns[:8])
value_vars=list(dataset_modified.columns[8:])

In [21]:
value_vars

['Question 1-Response',
 'Question 2-Response',
 'Question 3-Open-Ended Response',
 'Question 4-Response',
 'Question 4-Other (please specify)',
 'Question 5-Response 1',
 'Question 5-Response 2',
 'Question 5-Response 3',
 'Question 5-Response 4',
 'Question 5-Response 5',
 'Question 5-Response 6',
 'Question 6-Response 1',
 'Question 6-Response 2',
 'Question 6-Response 3',
 'Question 6-Response 4',
 'Question 6-Response 5',
 'Question 6-Response 6',
 'Question 7-Response 1',
 'Question 7-Unscheduled',
 'Question 8-Response 1',
 'Question 8-Response 2',
 'Question 8-Response 3',
 'Question 8-Response 4',
 'Question 9-Response 1',
 'Question 9-Response 2',
 'Question 9-Response 3',
 'Question 9-Response 4',
 'Question 10-Response 1',
 'Question 10-Response 2',
 'Question 10-Response 3',
 'Question 10-Response 4',
 'Question 10-Response 5',
 'Question 11-Reponse 1',
 'Question 11-Response 2',
 'Question 12-Response',
 'Question 13-Response',
 'Question 14-Response',
 'Question 15-Respo

In [48]:
dataset_melted=pd.melt(dataset_modified,id_vars=id_vars,value_vars=value_vars,var_name='Question + Subquestion',value_name='Answer')
dataset_melted.sort_values(by=['Respondent ID','Question + Subquestion'])

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question + Subquestion,Answer
181,1001882880,Infrastructure,NaN,Staff,Millennial (born between 1981-2000),Female,0-2 years,Full time Employee,Question 1-Response,Answer 7
5527,1001882880,Infrastructure,NaN,Staff,Millennial (born between 1981-2000),Female,0-2 years,Full time Employee,Question 10-Response 1,NaN
5725,1001882880,Infrastructure,NaN,Staff,Millennial (born between 1981-2000),Female,0-2 years,Full time Employee,Question 10-Response 2,Answer 8
5923,1001882880,Infrastructure,NaN,Staff,Millennial (born between 1981-2000),Female,0-2 years,Full time Employee,Question 10-Response 3,Answer 4
6121,1001882880,Infrastructure,NaN,Staff,Millennial (born between 1981-2000),Female,0-2 years,Full time Employee,Question 10-Response 4,NaN
...,...,...,...,...,...,...,...,...,...,...
4438,9983361274,Information Technology,NaN,Staff,Generation X (born between 1965-1980),Male,5-10 years,Full time Employee,Question 8-Response 4,Answer 8
4636,9983361274,Information Technology,NaN,Staff,Generation X (born between 1965-1980),Male,5-10 years,Full time Employee,Question 9-Response 1,NaN
4834,9983361274,Information Technology,NaN,Staff,Generation X (born between 1965-1980),Male,5-10 years,Full time Employee,Question 9-Response 2,NaN
5032,9983361274,Information Technology,NaN,Staff,Generation X (born between 1965-1980),Male,5-10 years,Full time Employee,Question 9-Response 3,NaN


In [29]:
questions_import=pd.read_excel("Survey_Data.xlsx",sheet_name="Question")
questions_import

,Raw Question,Raw Subquestion,Question,Subquestion,Question + Subquestion
0,Respondent ID,NaN,Respondent ID,NaN,Respondent ID
1,Start Date,NaN,Start Date,NaN,Start Date
2,End Date,NaN,End Date,NaN,End Date
3,Email Address,NaN,Email Address,NaN,Email Address
4,First Name,NaN,First Name,NaN,First Name
...,...,...,...,...,...
95,NaN,Response 13,Question 29,Response 13,Question 29 - Response 13
96,NaN,Response 14,Question 29,Response 14,Question 29 - Response 14
97,Question 30,Response 1,Question 30,Response 1,Question 30 - Response 1
98,NaN,Response 2,Question 30,Response 2,Question 30 - Response 2


In [32]:
questions=questions_import.copy()
questions.drop(columns=["Raw Question","Raw Subquestion","Subquestion"],axis=1,inplace=True)
questions.dropna(inplace=True)
questions

,Question,Question + Subquestion
0,Respondent ID,Respondent ID
1,Start Date,Start Date
2,End Date,End Date
3,Email Address,Email Address
4,First Name,First Name
...,...,...
95,Question 29,Question 29 - Response 13
96,Question 29,Question 29 - Response 14
97,Question 30,Question 30 - Response 1
98,Question 30,Question 30 - Response 2


In [41]:
dataset_merged=pd.merge(left=dataset_melted,right=questions,how="left",left_on="Question + Subquestion", right_on="Question + Subquestion")
print("Original Data:",len(dataset_melted))
print("Merged Data:",len(dataset_merged))
dataset_merged

Original Data: 17028
Merged Data: 17028


,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question + Subquestion,Answer,Question
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1-Response,NaN,NaN
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1-Response,Answer 4,NaN
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1-Response,Answer 5,NaN
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1-Response,Answer 1,NaN
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1-Response,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,NaN
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,NaN
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,NaN
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,NaN


In [57]:
dataset_melted.rename(columns={"Question + Subquestion":"Question_Subquestion"},inplace=True)
questions.rename(columns={"Question + Subquestion":"Question_Subquestion"},inplace=True)
dataset_merged=pysqldf(""" select dataset_melted.*,questions.Question from dataset_melted left join questions on dataset_melted.Question_Subquestion=questions.Question_Subquestion;""" )

In [58]:
dataset_merged["Question"]=dataset_melted['Question_Subquestion'].str.split('-').str[0]
dataset_merged

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question_Subquestion,Answer,Question
0,5379192392,Infrastructure,None,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1-Response,None,Question 1
1,2658722536,Finance,None,Staff,None,None,10+ years,Full time Employee,Question 1-Response,Answer 4,Question 1
2,4044163394,Infrastructure,None,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1-Response,Answer 5,Question 1
3,5535865599,Infrastructure,None,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1-Response,Answer 1,Question 1
4,3356802928,Port Operations,None,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1-Response,None,Question 1
...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,None,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30
17024,5157705612,Finance,None,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30
17025,9920755555,Port Operations,None,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,None,Question 30
17026,6638341389,Infrastructure,None,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,None,Question 30


In [61]:
dataset_merged.rename(columns={"Respondent ID":"Respondent_ID"},inplace=True)
respondents=dataset_merged[dataset_merged["Answer"].notna()]
respondents=pysqldf(""" 
select Question,Count(Respondent_ID) as Respondents from respondents
group by Question;
""")  

In [64]:
dataset_merged_two=pysqldf(""" select dataset_merged.*,respondents.Respondents from dataset_merged left join respondents on dataset_merged.Question=respondents.Question;""" )
dataset_merged_two

,Respondent_ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question_Subquestion,Answer,Question,Respondents
0,5379192392,Infrastructure,None,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1-Response,None,Question 1,119
1,2658722536,Finance,None,Staff,None,None,10+ years,Full time Employee,Question 1-Response,Answer 4,Question 1,119
2,4044163394,Infrastructure,None,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1-Response,Answer 5,Question 1,119
3,5535865599,Infrastructure,None,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1-Response,Answer 1,Question 1,119
4,3356802928,Port Operations,None,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1-Response,None,Question 1,119
...,...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,None,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30,332
17024,5157705612,Finance,None,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30,332
17025,9920755555,Port Operations,None,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,None,Question 30,332
17026,6638341389,Infrastructure,None,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,None,Question 30,332


In [65]:
dataset_merged_two.to_excel("Output.xlsx",index=False)